In [1]:
import pandas as pd


In [28]:
cols =  [{'NAME': 'FIPS_STATE', 'START': 0, 'STOP': 2, 'LENGTH': 2},
         {'NAME': 'FIPS_COUNTY', 'START': 5, 'STOP': 8, 'LENGTH': 3},
         {'NAME': 'COUNTY_NAME', 'START': 9, 'STOP': 40, 'LENGTH': 30},
         # Unknowns cols
         
         {'NAME': 'COUNTY_LAT', 'START': 121, 'STOP': 131, 'LENGTH': 9},
         {'NAME': 'COUNTY_LONG', 'START': 131, 'STOP': 142, 'LENGTH': 10},
        ]
# {'NAME': 'SEQUENCE_NUMBER', 'START': 53, 'STOP': 60, 'LENGTH': 7}]

In [29]:
filename = 'C:/data/HMDA/Census/1990/Gazetteer/counties.txt'
     


In [55]:
def readDataFile(filename):
    
    df = pd.read_csv(filename, header=None, delimiter="\t", names=["original"])

    for col in cols:
        #print(col['NAME'])
        df[col['NAME']] = df.original.map(lambda x: x[col["START"]:col["STOP"]])

    df = df.drop(['original'], axis=1)
    
    
    return df

In [56]:
df = readDataFile(filename)
df['COUNTY_NAME'] = df['COUNTY_NAME'].str.strip()

df['COUNTY_LAT'] = df['COUNTY_LAT'].astype(float)
df['COUNTY_LONG'] = df['COUNTY_LONG'].astype(float)
df['COUNTY_LAT'] = df['COUNTY_LAT'] / 1000000
df['COUNTY_LONG'] = df['COUNTY_LONG'] / 1000000
df.head()

,FIPS_STATE,FIPS_COUNTY,COUNTY_NAME,COUNTY_LAT,COUNTY_LONG
0,02,013,Aleutians East Borough,55.229183,-161.915191
1,02,016,Aleutians West Census Area,52.242645,-175.129755
2,02,020,Anchorage Borough,61.178368,-149.186416
3,02,050,Bethel Census Area,60.939161,-160.147317
4,02,060,Bristol Bay Borough,58.744623,-156.988097


In [58]:
df.to_csv('counties.csv', index=False)

In [59]:
df[df['COUNTY_NAME'].isin(['Plymouth County'])]

,FIPS_STATE,FIPS_COUNTY,COUNTY_NAME,COUNTY_LAT,COUNTY_LONG
620,19,149,Plymouth County,42.736720,-96.216082
1183,25,023,Plymouth County,41.990716,-70.737379


# Upload to DB

In [88]:
sql = "LOAD DATA INFILE 'C:/Users/Matt/git/hmda/counties.csv' INTO TABLE hmda.counties FIELDS TERMINATED BY ',' ENCLOSED BY \'\"\' LINES TERMINATED BY '\\n' IGNORE 1 ROWS (FIPS_STATE,FIPS_COUNTY,COUNTY_NAME,COUNTY_LAT,COUNTY_LONG)"

import pymysql


# MySQL Parameters
DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASS = ''
DB_NAME = 'hmda'

# Files location
folder = "c:/data/HMDA/"

connection = pymysql.connect(host=DB_HOST,
                             user=DB_USER,
                             password=DB_PASS,
                             db=DB_NAME,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)


#print(sql)
try:

    with connection.cursor() as cursor:
        result_row = cursor.execute(sql)
        connection.commit()

    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT count(*) FROM counties"
        cursor.execute(sql)
        result = cursor.fetchone()
        print(result)
finally:
    connection.close()

{'count(*)': 3141}


C:\Users\Matt\AppData\Local\conda\conda\envs\lsci-gpu\lib\site-packages\pymysql\cursors.py:329: Warning: (1265, "Data truncated for column 'COUNTY_LONG' at row 1")
  self._do_get_result()
C:\Users\Matt\AppData\Local\conda\conda\envs\lsci-gpu\lib\site-packages\pymysql\cursors.py:329: Warning: (1265, "Data truncated for column 'COUNTY_LONG' at row 2")
  self._do_get_result()
C:\Users\Matt\AppData\Local\conda\conda\envs\lsci-gpu\lib\site-packages\pymysql\cursors.py:329: Warning: (1265, "Data truncated for column 'COUNTY_LONG' at row 3")
  self._do_get_result()
C:\Users\Matt\AppData\Local\conda\conda\envs\lsci-gpu\lib\site-packages\pymysql\cursors.py:329: Warning: (1265, "Data truncated for column 'COUNTY_LONG' at row 4")
  self._do_get_result()
C:\Users\Matt\AppData\Local\conda\conda\envs\lsci-gpu\lib\site-packages\pymysql\cursors.py:329: Warning: (1265, "Data truncated for column 'COUNTY_LONG' at row 5")
  self._do_get_result()
C:\Users\Matt\AppData\Local\conda\conda\envs\lsci-gpu\lib\si

In [87]:
print(sql)

LOAD DATA INFILE 'C:\Users\Matt\git\hmda\counties.csv' INTO TABLE hmda.counties FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n' IGNORE 1 ROWS (FIPS_STATE,FIPS_COUNTY,COUNTY_NAME,COUNTY_LAT,COUNTY_LONG)
